In [0]:
from pyspark.sql import functions as F

In [0]:
# Ler da bronze
df_bronze = spark.table("bronze.data_suicide.who_suicide_data")

# Selecionar os campos para a silver
df_silver = (
    df_bronze
    .withColumn("country_code", F.col("SpatialDim"))
    .withColumn("region", F.col("ParentLocation"))
    .withColumn("year", F.col("TimeDim"))
    .withColumn(
        "sex",
        F.when(F.col("Dim1") == "SEX_MLE", "male")
         .when(F.col("Dim1") == "SEX_FMLE", "female")
         .when(F.col("Dim1") == "SEX_BTSX", "both")
         .otherwise(F.col("Dim1"))
    )
    .withColumn("rate_per_100k", F.col("NumericValue"))
    .withColumn("ci_low", F.col("Low"))
    .withColumn("ci_high", F.col("High"))
    .withColumn("value_label", F.col("Value"))
    .select("country_code", "region", "year", "sex",
            "rate_per_100k", "ci_low", "ci_high", "value_label")
)

spark.sql(f"CREATE SCHEMA IF NOT EXISTS silver.data_suicide")

# Salvar como tabela silver
(
    df_silver.write
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable("silver.data_suicide.who_suicide_data")
)


In [0]:

# Adicionar comentários na tabela e colunas
spark.sql("""
COMMENT ON TABLE silver.data_suicide.who_suicide_data IS
'Tabela Silver com taxas de suicídio (SDG 3.4.2) por país/ano/sexo, derivada da camada bronze WHO GHO API'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.country_code IS
'Código ISO-3 do país, conforme OMS (SpatialDim)'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.region IS
'Região da OMS associada ao país (ParentLocation)'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.year IS
'Ano de referência do dado (TimeDim)'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.sex IS
'Sexo (male/female/both), mapeado de Dim1'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.rate_per_100k IS
'Taxa de suicídio por 100.000 habitantes (NumericValue)'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.ci_low IS
'Limite inferior do intervalo de confiança'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.ci_high IS
'Limite superior do intervalo de confiança'
""")

spark.sql("""
COMMENT ON COLUMN silver.data_suicide.who_suicide_data.value_label IS
'Texto original da OMS com valor + IC'
""")


In [0]:
display(spark.sql("DESCRIBE TABLE silver.data_suicide.who_suicide_data"))

In [0]:
display(spark.sql("SELECT * FROM silver.data_suicide.who_suicide_data"))